In [50]:
# En primer lugar se instalan e importan las librerías y dependencias que serán utilizadas en el proceso.
!pip install SPARQLWrapper  
!pip install wikipedia      
import pandas as pd
from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON
import wikipedia
import json
import xml.etree.ElementTree

In [62]:
print("¿Cuál es el identificador de la clase de la cual se quiere generar un diagrama?")
clase = input()

¿Cuál es el identificador de la clase de la cual se quiere generar un diagrama?
Q349


In [63]:
#Se define la función para consultar Wikidata. Se consulta el endpoint señalado y 
#devuelve los resultados como un DataFrame de Pandas.

def query_wikidata(sparql_query, sparql_service_url):        
  
    # crear la conexión con el endpoint
    sparql = SPARQLWrapper(sparql_service_url, agent="Estoy replicando el Mapa de Comics. Mi correo es{}".format(mi_correo))    
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # solicitar resultados
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

In [64]:
#Aquí se genera la consulta. La primera es para obtener el número de instancias de la clase a representar
sparql_query1 = """SELECT DISTINCT ?clase ?claseLabel
WHERE {
#occupation (P106) must be Formula 1 driver (Q10841764)
?clase wdt:P31 wd:"""+clase+""".
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}
"""
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
instancias_wikidata = query_wikidata(sparql_query1, sparql_service_url)

In [65]:
#Se confirma que la consulta haya salido bien
instancias_wikidata.head()

,clase.type,clase.value,claseLabel.type,claseLabel.value,claseLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q581356,literal,Q581356,NaN
1,uri,http://www.wikidata.org/entity/Q627949,literal,Q627949,NaN
2,uri,http://www.wikidata.org/entity/Q638269,literal,Q638269,NaN
3,uri,http://www.wikidata.org/entity/Q895138,literal,Q895138,NaN
4,uri,http://www.wikidata.org/entity/Q907328,literal,Q907328,NaN


In [66]:
numero_de_instancias = instancias_wikidata.shape[0]
numero_de_instancias

201

In [67]:
#Aquí se genera la consulta. Esta se hace para obtener las propiedades de la clase.
sparql_query2 = """SELECT DISTINCT ?p ?pLabel
WHERE {
#Instancia de (P31) que son comics (Q1004)
?comic wdt:P31 wd:"""+clase+""";
?p ?x.
#to obtain property labels:
?property wikibase:directClaim ?p
#to obtain labels in spanish:
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}   
    """
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
propiedades_wikidata = query_wikidata(sparql_query2, sparql_service_url)

In [88]:
#Confirmamos que la consulta haya salido bien
propiedades_wikidata.head()

,p.type,p.value,pLabel.type,pLabel.value
0,uri,http://www.wikidata.org/prop/direct/P279,literal,http://www.wikidata.org/prop/direct/P279
1,uri,http://www.wikidata.org/prop/direct/P276,literal,http://www.wikidata.org/prop/direct/P276
2,uri,http://www.wikidata.org/prop/direct/P349,literal,http://www.wikidata.org/prop/direct/P349
3,uri,http://www.wikidata.org/prop/direct/P373,literal,http://www.wikidata.org/prop/direct/P373
4,uri,http://www.wikidata.org/prop/direct/P495,literal,http://www.wikidata.org/prop/direct/P495


In [104]:
lista_completa = propiedades_wikidata["p.value"].tolist()
lista_propiedades = []
for ele in lista_completa:
    ele = ele[36:]
    lista_propiedades.append(ele)
lista_propiedades

['P279',
 'P276',
 'P349',
 'P373',
 'P495',
 'P571',
 'P646',
 'P641',
 'P664',
 'P691',
 'P910',
 'P935',
 'P1014',
 'P1051',
 'P1245',
 'P1296',
 'P1417',
 'P1872',
 'P1873',
 'P1889',
 'P2167',
 'P2283',
 'P2347',
 'P2671',
 'P2910',
 'P2924',
 'P2959',
 'P3095',
 'P3221',
 'P3417',
 'P3509',
 'P3827',
 'P4613',
 'P4854',
 'P5160',
 'P5250',
 'P5429',
 'P6104',
 'P6262',
 'P7033',
 'P7749',
 'P7818',
 'P8309',
 'P8408',
 'P8785',
 'P18',
 'P17',
 'P31',
 'P227',
 'P527',
 'P585',
 'P740',
 'P989',
 'P2541',
 'P4212',
 'P5513',
 'P6366',
 'P244',
 'P361',
 'P452',
 'P582',
 'P580',
 'P856',
 'P2013',
 'P3338',
 'P186',
 'P460',
 'P488',
 'P508',
 'P1269',
 'P1705',
 'P138',
 'P797',
 'P1132',
 'P1350',
 'P1807',
 'P2348',
 'P3450',
 'P4235',
 'P5082',
 'P407',
 'P443',
 'P625',
 'P937',
 'P1433',
 'P2002',
 'P2003',
 'P2397',
 'P2596',
 'P3259',
 'P5916',
 'P127',
 'P159',
 'P421',
 'P461',
 'P2521',
 'P2561',
 'P2581',
 'P10',
 'P21',
 'P131',
 'P170',
 'P1282',
 'P6766']

In [109]:
#Obtenemos de las propiedades anteriores aquellas que son object properties. Se almacenan en una lista.
object_properties = []
#for ele in lista_propiedades:
    #Ver si es object property, en cuyo caso introducir en la lista object_properties
    # If ele es object property then
    #       object_properties.append(ele)
#object_properties

In [70]:
#Aquí generamos la consulta. Se realiza esta consulta para obtener las clases que pertenecen a dicha propiedad. Se realiza esta
# consulta para todas las object properties encontradas anteriormente.
propiedad = "P495"
sparql_query3 = """SELECT DISTINCT ?comic ?comicLabel ?country ?countryLabel ?Clase ?ClaseLabel
WHERE {
#occupation (P106) must be Formula 1 driver (Q10841764)
?comic wdt:P31 wd:"""+clase+""" .
?comic wdt:"""+propiedad +""" ?country .
?country wdt:P31 ?Clase .
#to obtain labels in spanish:
SERVICE wikibase:label { bd:serviceParam wikibase:language "es". }
}  
"""
#para consultar otro endpoint, cambiar la URL del servicio y la consulta.
sparql_service_url = "https://query.wikidata.org/sparql"
clases_object_property_wikidata = query_wikidata(sparql_query3, sparql_service_url)

In [71]:
#Confirmamos que la consulta haya salido bien
clases_object_property_wikidata.head()

,Clase.type,Clase.value,ClaseLabel.type,ClaseLabel.value,ClaseLabel.xml:lang,comic.type,comic.value,comicLabel.type,comicLabel.value,comicLabel.xml:lang,country.type,country.value,countryLabel.type,countryLabel.value,countryLabel.xml:lang
0,uri,http://www.wikidata.org/entity/Q1520223,literal,república constitucional,es,uri,http://www.wikidata.org/entity/Q76851344,literal,Q76851344,NaN,uri,http://www.wikidata.org/entity/Q252,literal,Indonesia,es
1,uri,http://www.wikidata.org/entity/Q3336843,literal,nación constitutiva del Reino Unido,es,uri,http://www.wikidata.org/entity/Q681142,literal,planchado extremo,es,uri,http://www.wikidata.org/entity/Q21,literal,Inglaterra,es
2,uri,http://www.wikidata.org/entity/Q41614,literal,monarquía constitucional,es,uri,http://www.wikidata.org/entity/Q3641667,literal,Q3641667,NaN,uri,http://www.wikidata.org/entity/Q424,literal,Camboya,es
3,uri,http://www.wikidata.org/entity/Q3624078,literal,Estado soberano,es,uri,http://www.wikidata.org/entity/Q7159160,literal,Pedestrianismo,es,uri,http://www.wikidata.org/entity/Q145,literal,Reino Unido,es
4,uri,http://www.wikidata.org/entity/Q6256,literal,país,es,uri,http://www.wikidata.org/entity/Q2533407,literal,salvamento y socorrismo,es,uri,http://www.wikidata.org/entity/Q408,literal,Australia,es


In [74]:
#Aqui se obtienen el numero de apariciones de cada clase para hacer un recuento y obtener la clase a representar
clases_object_property_wikidata["country.value"].value_counts()

http://www.wikidata.org/entity/Q252      6
http://www.wikidata.org/entity/Q28025    5
http://www.wikidata.org/entity/Q145      5
http://www.wikidata.org/entity/Q408      5
http://www.wikidata.org/entity/Q664      4
http://www.wikidata.org/entity/Q33       4
http://www.wikidata.org/entity/Q424      3
http://www.wikidata.org/entity/Q18097    3
http://www.wikidata.org/entity/Q836      2
http://www.wikidata.org/entity/Q39       1
http://www.wikidata.org/entity/Q21       1
http://www.wikidata.org/entity/Q34       1
Name: country.value, dtype: int64

In [ ]:
# Se van comparando cada clase de mayor numero de apariciones a menor numero de apariciones con las clases del value type 
# constraint de la pripiedad de la que se ha realizado la consulta. De esta manera obtendremos aquella clase a representar 
# en el diagrama
